In [ ]:
from wikidata.client import Client
import urllib
proxies = {"http": "http://127.0.0.1:7890", "https": "http://127.0.0.1:7890"}
# 设置代理
handler = urllib.request.ProxyHandler(proxies)
opener = urllib.request.build_opener(handler)
client = Client(opener=opener)


In [ ]:
from tag_utils import Annotate, Annotation_mentions

txt = 'The wine list is excellent .'
obj = Annotation_mentions(txt)
for i in obj.keys():
    print(i + "  " + obj[i])
print("=" * 30)
obj = Annotate(txt, theta=0.05)
for k in obj:
    print(obj[k][2], " ---> ", obj[k][1], obj[k][0], obj[k][-1])


In [ ]:
from qwikidata.entity import WikidataItem, WikidataLexeme, WikidataProperty
from qwikidata.linked_data_interface import get_entity_dict_from_api

# create an item representing "Douglas Adams"
Q_DOUGLAS_ADAMS = "Q42"
q42_dict = get_entity_dict_from_api(Q_DOUGLAS_ADAMS)
q42 = WikidataItem(q42_dict)

# create a property representing "subclass of"
P_SUBCLASS_OF = "P279"
p279_dict = get_entity_dict_from_api(P_SUBCLASS_OF)
p279 = WikidataProperty(p279_dict)

# create a lexeme representing "bank"
L_BANK = "L3354"
l3354_dict = get_entity_dict_from_api(L_BANK)
l3354 = WikidataLexeme(l3354_dict)

In [ ]:
from qwikidata.sparql import return_sparql_query_results, get_subclasses_of_item
# send any sparql query to the wikidata query service and get full result back
# here we use an example that counts the number of humans
qid = "Q730298"
sparql_query = """
SELECT ?pLabel WHERE {{
  wd:{} wdt:P279 ?p .
  SERVICE wikibase:label {{
    bd:serviceParam wikibase:language "en" .
   }}
}}
""".format(qid)
results = return_sparql_query_results(sparql_query)
uris = [binding["pLabel"]["value"] for binding in results["results"]["bindings"]]
uris

In [ ]:
from qwikidata.sparql import get_subclasses_of_item
# use convenience function to get subclasses of an item as a list of item ids
Q_RIVER = "Q214276"
get_subclasses_of_item(Q_RIVER)

In [ ]:
from nltk.corpus import wordnet as wn

hyper = lambda s: s.hypernyms()
for domain in ("laptop",):
    for synset in wn.synsets(domain, pos=wn.NOUN):
        print(synset.hyponyms())
        print(list(synset.closure(hyper)))
    print('\n')

In [ ]:
import requests
import tagme
for ann in tagme.annotate("MacBook Pro").get_annotations(0.1):
    # json = requests.get(
    # "https://en.wikipedia.org/w/api.php?action=query&format=json&prop=pageprops&titles=MacBook Pro"
    # ).json()
    # wikidata_id = json['query']['pages'][f'{ann.entity_id}']['pageprops']['wikibase_item']
    print(ann.uri(), ann.entity_id, ann.entity_title, ann.mention)

In [ ]:
import sys
sys.path.append("../")
from eval import absa_evaluate, evaluate
from tag_utils import ot2bio_absa
text = [
    line.split("***")[0]
    for line in open("/root/graduation/processed/dp_tmp/laptop.train.txt").read().splitlines()
]
gold_Y = [
    ot2bio_absa(line.split("***")[1].split())
    for line in open("/root/graduation/processed/dp_tmp/laptop.train.txt").read().splitlines()
]

pred_Y = [
    ot2bio_absa(line.split("***")[-1].split())
    for line in open("/root/graduation/processed/dp_tmp/laptop.train.txt").read().splitlines()
]
pred_Y_ = []
gold_Y_ = []
for idx, (pred, gold) in enumerate(zip(pred_Y, gold_Y)):
    sentence = text[idx]
    # res = Annotate(sentence, theta=0.05)
    if not all(item == 'O' for item in pred):
        pred_Y_.append(pred)
        gold_Y_.append(gold)
absa_evaluate(pred_Y_, gold_Y_), evaluate(pred_Y_, gold_Y_)

In [3]:
import fasttext

model = fasttext.load_model('../cc.en.300.bin')

In [6]:
import pickle
from collections import Counter
from operator import itemgetter
from typing import List, Tuple

import nltk
import numpy as np
import torch
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity
domain = 'service'
nltk.download('stopwords')

wnl = WordNetLemmatizer()
with open(f"./entities/{domain}.pkl", "rb") as f:
    entities: List[str] = pickle.load(f)
# remove stopwords
sets = stopwords.words('english')
# lemmatization
entities = [wnl.lemmatize(word, 'n').lower() for word in entities if word not in sets]
counters = Counter(entities)
sorted_entities: List[Tuple[str, int]] = sorted(filter(lambda item: item[1] >= 5, counters.items()),
                                                key=lambda item: item[1],
                                                reverse=True)

vec_dict = {}
for entity in sorted_entities:
    e = entity[0]
    vec_dict[e] = model.get_word_vector(e)


getter = itemgetter(*[entity for entity, _ in sorted_entities[:10]])
mean_vec = np.average(getter(vec_dict),
                      axis=0,
                      weights=[count for _, count in sorted_entities[:10]])
np.save(f"{domain}_mean_vec", mean_vec)
res = np.array([
    cosine_similarity(mean_vec.reshape(1, -1), vec_dict[k].reshape(1, -1)) for k in vec_dict
]).squeeze()
topk = torch.topk(torch.from_numpy(res), res.shape[0]).indices.tolist()
itemgetter(*topk)(sorted_entities), sorted_entities

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


((('uop', 121),
  ('egroups', 234),
  ('etrade', 90),
  ('devry', 22),
  ('egroup', 5),
  ('one', 73),
  ('yahoo', 44),
  ('alot', 5),
  ('time', 99),
  ('ipo', 6),
  ('day', 8),
  ('ameritrade', 6),
  ('thing', 10),
  ('ctu', 63),
  ('you', 20),
  ('online', 62),
  ('ipos', 10),
  ('server', 8),
  ('u', 12),
  ('group', 70),
  ('read', 21),
  ('service', 84),
  ('post', 16),
  ('week', 13),
  ('good', 71),
  ('write', 6),
  ('money', 39),
  ('way', 14),
  ('internet', 5),
  ('hour', 10),
  ('account', 55),
  ('number', 7),
  ('people', 85),
  ('news', 6),
  ('if', 15),
  ('web', 12),
  ('groups', 9),
  ('start', 6),
  ('now', 11),
  ('getting', 5),
  ('check', 8),
  ('com', 9),
  ('chat', 12),
  ('something', 5),
  ('instruction', 5),
  ('paypal', 17),
  ('think', 30),
  ('trading', 24),
  ('mailing', 6),
  ('class', 66),
  ('learning', 17),
  ('school', 54),
  ('list', 45),
  ('lot', 23),
  ('join', 13),
  ('thought', 11),
  ('nice', 24),
  ('always', 9),
  ('page', 11),
  ('free', 3

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(mean_vec.reshape(1, -1),
                  np.array(model.get_word_vector("the staff".lower())).reshape(1, -1))

In [ ]:
from sklearn.cluster import KMeans
import numpy as np
from sklearn.metrics import calinski_harabasz_score

max_score, clusters = 0, 0

X = np.array(list(vec_dict.values()))
for i in range(2, 100):
    kmeans = KMeans(n_clusters=i).fit(X, sample_weight=[entity[1] for entity in sorted_entities[::-1]])
    score = calinski_harabasz_score(X, kmeans.labels_)
    if score > max_score:
        max_score = score
        clusters = i
max_score, clusters

In [ ]:
kmeans = KMeans(n_clusters=clusters).fit(X, sample_weight=[entity[1] for entity in sorted_entities[::-1]])

In [ ]:
Counter(kmeans.labels_)

In [ ]:
np.asarray(list(vec_dict.keys()))[kmeans.labels_ == 1]